# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [25]:
# Calculating each column separately
total_schools = school_data_complete['school_name'].nunique()
total_students = school_data_complete["Student ID"].count()
total_budget = school_data_complete[["school_name","budget"]].drop_duplicates()["budget"].sum()
avg_math_score = school_data_complete["math_score"].mean()
avg_reading_score = school_data_complete["reading_score"].mean()
pct_pass_math = school_data_complete.loc[school_data_complete["math_score"] >= 70,:]["math_score"].count() / school_data_complete["math_score"].count()*100
pct_pass_reading = school_data_complete.loc[school_data_complete["reading_score"] >= 70,:]["reading_score"].count() / school_data_complete["reading_score"].count()*100
overall_pass_rate = (avg_math_score+avg_reading_score)/2

# Converting each result into a dataframe
results = pd.DataFrame(np.column_stack([total_schools, total_students, total_budget, avg_math_score, avg_reading_score, pct_pass_math, pct_pass_reading, overall_pass_rate]), 
                               columns=['Total Schools','Total Students','Total Budget','Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing Rate'])

# Formatting to columns
results["Total Schools"] = int(results["Total Schools"])
results["Total Students"] = int(results["Total Students"])
results["Total Students"] = results["Total Students"].map("{:,}".format)
results["Total Budget"] = int(results["Total Budget"])
results["Total Budget"] = results["Total Budget"].map("${:,}".format)
results["Average Math Score"] = results["Average Math Score"].map("{:.2f}".format)
results["Average Reading Score"] = results["Average Reading Score"].map("{:.2f}".format)
results["% Passing Math"] = results["% Passing Math"].map("{:.2f}%".format)
results["% Passing Reading"] = results["% Passing Reading"].map("{:.2f}%".format)
results["% Overall Passing Rate"] = results["% Overall Passing Rate"].map("{:.2f}%".format)

# Showing the final result
results

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428",78.99,81.88,74.98%,85.81%,80.43%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [26]:
# Calculating every column separately
type_school = school_data_complete[["school_name","type"]].drop_duplicates().set_index('school_name').rename(columns = {
    'type': "School Type" })
total_student = pd.DataFrame(school_data_complete.groupby('school_name')['student_name'].count()).rename(columns = {
    'student_name': "Total Students" })
total_school_budget = school_data_complete[["school_name","budget"]].drop_duplicates().set_index('school_name').rename(columns = {
    'budget': "Total School Budget" })
avg_math_score = pd.DataFrame(school_data_complete.groupby('school_name')['math_score'].mean()).rename(columns = {
    'math_score': "Average Math Score" })
avg_reading_score = pd.DataFrame(school_data_complete.groupby('school_name')['reading_score'].mean()).rename(columns = {
    'reading_score': "Average Reading Score" })
per_pass_math = pd.DataFrame(school_data_complete.loc[school_data_complete["math_score"] >= 70,
                                      :].groupby('school_name')["math_score"].count() / 
             school_data_complete.groupby('school_name')["student_name"].count()*100).rename(columns = {0: "% Passing Math" })

per_pass_reading = pd.DataFrame(school_data_complete.loc[school_data_complete["reading_score"] >= 70,
                                      :].groupby('school_name')["reading_score"].count() / 
             school_data_complete.groupby('school_name')["student_name"].count()*100).rename(columns = {0: "% Passing Reading" })

# Making a join between the columns into dataframe using index as union
school_summary = type_school.join([total_student,total_school_budget,avg_math_score,avg_reading_score,per_pass_math,per_pass_reading])

# Using the first columns of the union to create new ones
school_summary['Per Student Budget'] = school_summary["Total School Budget"] / school_summary["Total Students"]
school_summary['% Overall Passing Rate'] = (school_summary["% Passing Math"] + school_summary["% Passing Reading"])/2

# Ordering the columns
school_summary = school_summary[['School Type', 'Total Students', 'Total School Budget', 'Per Student Budget',
                                'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading',
                                '% Overall Passing Rate']].sort_values(by ='% Overall Passing Rate', ascending=False).head()

# Deleting index name just for no showing it in the result
del school_summary.index.name

# Formatting the table
school_summary["Total Students"] = school_summary["Total Students"].map("{:,}".format)
school_summary["Total School Budget"] = school_summary["Total School Budget"].map("${:,}".format)
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].astype(int)
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].map("${:,}".format)
school_summary["Average Math Score"] = school_summary["Average Math Score"].map("{:.2f}".format)
school_summary["Average Reading Score"] = school_summary["Average Reading Score"].map("{:.2f}".format)
school_summary["% Passing Math"] = school_summary["% Passing Math"].map("{:.2f}%".format)
school_summary["% Passing Reading"] = school_summary["% Passing Reading"].map("{:.2f}%".format)
school_summary["% Overall Passing Rate"] = school_summary["% Overall Passing Rate"].map("{:.2f}%".format)

# Showing the summary
school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.06,83.98,94.13%,97.04%,95.59%
Thomas High School,Charter,"1,635","$1,043,130",$638,83.42,83.85,93.27%,97.31%,95.29%
Pena High School,Charter,962,"$585,858",$609,83.84,84.04,94.59%,95.95%,95.27%
Griffin High School,Charter,"1,468","$917,500",$625,83.35,83.82,93.39%,97.14%,95.27%
Wilson High School,Charter,"2,283","$1,319,574",$578,83.27,83.99,93.87%,96.54%,95.20%


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [27]:
# Calculating every column separately
type_school = school_data_complete[["school_name","type"]].drop_duplicates().set_index('school_name').rename(columns = {
    'type': "School Type" })
total_student = pd.DataFrame(school_data_complete.groupby('school_name')['student_name'].count()).rename(columns = {
    'student_name': "Total Students" })
total_school_budget = school_data_complete[["school_name","budget"]].drop_duplicates().set_index('school_name').rename(columns = {
    'budget': "Total School Budget" })
avg_math_score = pd.DataFrame(school_data_complete.groupby('school_name')['math_score'].mean()).rename(columns = {
    'math_score': "Average Math Score" })
avg_reading_score = pd.DataFrame(school_data_complete.groupby('school_name')['reading_score'].mean()).rename(columns = {
    'reading_score': "Average Reading Score" })
per_pass_math = pd.DataFrame(school_data_complete.loc[school_data_complete["math_score"] >= 70,
                                      :].groupby('school_name')["math_score"].count() / 
             school_data_complete.groupby('school_name')["student_name"].count()*100).rename(columns = {0: "% Passing Math" })

per_pass_reading = pd.DataFrame(school_data_complete.loc[school_data_complete["reading_score"] >= 70,
                                      :].groupby('school_name')["reading_score"].count() / 
             school_data_complete.groupby('school_name')["student_name"].count()*100).rename(columns = {0: "% Passing Reading" })

# Making a join between the columns into dataframe using index as union
school_summary = type_school.join([total_student,total_school_budget,avg_math_score,avg_reading_score,per_pass_math,per_pass_reading])

# Using the first columns of the union to create new ones
school_summary['Per Student Budget'] = school_summary["Total School Budget"] / school_summary["Total Students"]
school_summary['% Overall Passing Rate'] = (school_summary["% Passing Math"] + school_summary["% Passing Reading"])/2

# Ordering the columns
school_summary = school_summary[['School Type', 'Total Students', 'Total School Budget', 'Per Student Budget',
                                'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading',
                                '% Overall Passing Rate']].sort_values(by ='% Overall Passing Rate').head()

# Deleting index name just for no showing it in the result
del school_summary.index.name

# Formatting the table
school_summary["Total Students"] = school_summary["Total Students"].map("{:,}".format)
school_summary["Total School Budget"] = school_summary["Total School Budget"].map("${:,}".format)
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].astype(int)
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].map("${:,}".format)
school_summary["Average Math Score"] = school_summary["Average Math Score"].map("{:.2f}".format)
school_summary["Average Reading Score"] = school_summary["Average Reading Score"].map("{:.2f}".format)
school_summary["% Passing Math"] = school_summary["% Passing Math"].map("{:.2f}%".format)
school_summary["% Passing Reading"] = school_summary["% Passing Reading"].map("{:.2f}%".format)
school_summary["% Overall Passing Rate"] = school_summary["% Overall Passing Rate"].map("{:.2f}%".format)

# Showing the summary
school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,"3,999","$2,547,363",$637,76.84,80.74,66.37%,80.22%,73.29%
Figueroa High School,District,"2,949","$1,884,411",$639,76.71,81.16,65.99%,80.74%,73.36%
Huang High School,District,"2,917","$1,910,635",$655,76.63,81.18,65.68%,81.32%,73.50%
Johnson High School,District,"4,761","$3,094,650",$650,77.07,80.97,66.06%,81.22%,73.64%
Ford High School,District,"2,739","$1,763,916",$644,77.10,80.75,68.31%,79.30%,73.80%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [33]:
# Calculating every column separately
avg_math_score_9thgrade = pd.DataFrame(school_data_complete.loc[school_data_complete["grade"] == "9th",
                                      :].groupby('school_name')["math_score"].mean()).rename(columns = {
    'math_score': "9th" })
avg_math_score_10thgrade = pd.DataFrame(school_data_complete.loc[school_data_complete["grade"] == "10th",
                                      :].groupby('school_name')["math_score"].mean()).rename(columns = {
    'math_score': "10th" })
avg_math_score_11thgrade = pd.DataFrame(school_data_complete.loc[school_data_complete["grade"] == "11th",
                                      :].groupby('school_name')["math_score"].mean()).rename(columns = {
    'math_score': "11th" })
avg_math_score_12thgrade = pd.DataFrame(school_data_complete.loc[school_data_complete["grade"] == "12th",
                                      :].groupby('school_name')["math_score"].mean()).rename(columns = {
    'math_score': "12th" })

# Making a join between the columns into dataframe using index as union
math_scores_by_grade = avg_math_score_9thgrade.join([avg_math_score_10thgrade,avg_math_score_11thgrade,avg_math_score_12thgrade])

# Deleting index name just for no showing it in the result
del math_scores_by_grade.index.name

# Formatting the table
math_scores_by_grade["9th"] = math_scores_by_grade["9th"].map("{:.2f}".format)
math_scores_by_grade["10th"] = math_scores_by_grade["10th"].map("{:.2f}".format)
math_scores_by_grade["11th"] = math_scores_by_grade["11th"].map("{:.2f}".format)
math_scores_by_grade["12th"] = math_scores_by_grade["12th"].map("{:.2f}".format)

# Showing the result
math_scores_by_grade

,9th,10th,11th,12th
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86
Pena High School,83.63,83.37,84.33,84.12


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [34]:
# Calculating every column separately
avg_reading_score_9thgrade = pd.DataFrame(school_data_complete.loc[school_data_complete["grade"] == "9th",
                                      :].groupby('school_name')["reading_score"].mean()).rename(columns = {
    'reading_score': "9th" })
avg_reading_score_10thgrade = pd.DataFrame(school_data_complete.loc[school_data_complete["grade"] == "10th",
                                      :].groupby('school_name')["reading_score"].mean()).rename(columns = {
    'reading_score': "10th" })
avg_reading_score_11thgrade = pd.DataFrame(school_data_complete.loc[school_data_complete["grade"] == "11th",
                                      :].groupby('school_name')["reading_score"].mean()).rename(columns = {
    'reading_score': "11th" })
avg_reading_score_12thgrade = pd.DataFrame(school_data_complete.loc[school_data_complete["grade"] == "12th",
                                      :].groupby('school_name')["reading_score"].mean()).rename(columns = {
    'reading_score': "12th" })

# Making a join between the columns into dataframe using index as union
reading_scores_by_grade = avg_math_score_9thgrade.join([avg_reading_score_10thgrade,avg_reading_score_11thgrade,avg_reading_score_12thgrade])

# Formatting the table
reading_scores_by_grade["9th"] = reading_scores_by_grade["9th"].map("{:.2f}".format)
reading_scores_by_grade["10th"] = reading_scores_by_grade["10th"].map("{:.2f}".format)
reading_scores_by_grade["11th"] = reading_scores_by_grade["11th"].map("{:.2f}".format)
reading_scores_by_grade["12th"] = reading_scores_by_grade["12th"].map("{:.2f}".format)

# Showing the result
reading_scores_by_grade

,9th,10th,11th,12th
Bailey High School,77.08,80.91,80.95,80.91
Cabrera High School,83.09,84.25,83.79,84.29
Figueroa High School,76.40,81.41,80.64,81.38
Ford High School,77.36,81.26,80.40,80.66
Griffin High School,82.04,83.71,84.29,84.01
Hernandez High School,77.44,80.66,81.40,80.86
Holden High School,83.79,83.32,83.82,84.70
Huang High School,77.03,81.51,81.42,80.31
Johnson High School,77.19,80.77,80.62,81.23
Pena High School,83.63,83.61,84.34,84.59


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [47]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["0 to 585", "585 to 615", "615 to 645", "645 to 675"]

In [48]:
# Creating the bins in the dataframe
school_data_complete['Spending Ranges (Per Student)'] = pd.cut(school_data_complete['budget']/school_data_complete['size'], spending_bins, labels = group_names)

# Calculating every column separately
avg_math_score = pd.DataFrame(school_data_complete.groupby('Spending Ranges (Per Student)')['math_score'].mean()).rename(columns = {
    'math_score': "Average Math Score" })
avg_reading_score = pd.DataFrame(school_data_complete.groupby('Spending Ranges (Per Student)')['reading_score'].mean()).rename(columns = {
    'reading_score': "Average Reading Score" })
per_pass_math = pd.DataFrame(school_data_complete.loc[school_data_complete["math_score"] >= 70,
                                      :].groupby('Spending Ranges (Per Student)')["math_score"].count() / 
             school_data_complete.groupby('Spending Ranges (Per Student)')["student_name"].count()*100).rename(columns = {
    0: "% Passing Math" })
per_pass_reading = pd.DataFrame(school_data_complete.loc[school_data_complete["reading_score"] >= 70,
                                      :].groupby('Spending Ranges (Per Student)')["reading_score"].count() / 
             school_data_complete.groupby('Spending Ranges (Per Student)')["student_name"].count()*100).rename(columns = {
    0: "% Passing Reading" })

# Making a join between the columns into dataframe using index as union
score_by_school_spreading = avg_math_score.join([avg_reading_score,per_pass_math,per_pass_reading])

# Using the first columns of the union to create new ones
score_by_school_spreading['% Overall Passing Rate'] = (score_by_school_spreading["% Passing Math"] + score_by_school_spreading["% Passing Reading"])/2

# Formatting the table
score_by_school_spreading["Average Math Score"] = score_by_school_spreading["Average Math Score"].map("{:.2f}".format)
score_by_school_spreading["Average Reading Score"] = score_by_school_spreading["Average Reading Score"].map("{:.2f}".format)
score_by_school_spreading["% Passing Math"] = score_by_school_spreading["% Passing Math"].map("{:.2f}%".format)
score_by_school_spreading["% Passing Reading"] = score_by_school_spreading["% Passing Reading"].map("{:.2f}%".format)
score_by_school_spreading["% Overall Passing Rate"] = score_by_school_spreading["% Overall Passing Rate"].map("{:.2f}%".format)

# Deleting index name just for no showing it in the result
del score_by_school_spreading.index.name

# Showing the result
score_by_school_spreading

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0 to 585,83.36,83.96,93.70%,96.69%,95.19%
585 to 615,83.53,83.84,94.12%,95.89%,95.01%
615 to 645,78.06,81.43,71.40%,83.61%,77.51%
645 to 675,77.05,81.01,66.23%,81.11%,73.67%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [49]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [51]:
# Creating the bins in the dataframe
school_data_complete['School Size'] = pd.cut(school_data_complete['size'], size_bins, labels = group_names)

# Calculating every column separately
avg_math_score = pd.DataFrame(school_data_complete.groupby('School Size')['math_score'].mean()).rename(columns = {
    'math_score': "Average Math Score" })
avg_reading_score = pd.DataFrame(school_data_complete.groupby('School Size')['reading_score'].mean()).rename(columns = {
    'reading_score': "Average Reading Score" })
per_pass_math = pd.DataFrame(school_data_complete.loc[school_data_complete["math_score"] >= 70,
                                      :].groupby('School Size')["math_score"].count() / 
             school_data_complete.groupby('School Size')["student_name"].count()*100).rename(columns = {
    0: "% Passing Math" })
per_pass_reading = pd.DataFrame(school_data_complete.loc[school_data_complete["reading_score"] >= 70,
                                      :].groupby('School Size')["reading_score"].count() / 
             school_data_complete.groupby('School Size')["student_name"].count()*100).rename(columns = {
    0: "% Passing Reading" })

# Making a join between the columns into dataframe using index as union
score_by_school_size = avg_math_score.join([avg_reading_score,per_pass_math,per_pass_reading])

# Using the first columns of the union to create new ones
score_by_school_size['% Overall Passing Rate'] = (score_by_school_size["% Passing Math"] + score_by_school_size["% Passing Reading"])/2

# Formatting the table
score_by_school_size["Average Math Score"] = score_by_school_size["Average Math Score"].map("{:.2f}".format)
score_by_school_size["Average Reading Score"] = score_by_school_size["Average Reading Score"].map("{:.2f}".format)
score_by_school_size["% Passing Math"] = score_by_school_size["% Passing Math"].map("{:.2f}%".format)
score_by_school_size["% Passing Reading"] = score_by_school_size["% Passing Reading"].map("{:.2f}%".format)
score_by_school_size["% Overall Passing Rate"] = score_by_school_size["% Overall Passing Rate"].map("{:.2f}%".format)

# Deleting index name just for no showing it in the result
del score_by_school_size.index.name

# Showing the result
score_by_school_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Small (<1000),83.83,83.97,93.95%,96.04%,95.00%
Medium (1000-2000),83.37,83.87,93.62%,96.77%,95.19%
Large (2000-5000),77.48,81.20,68.65%,82.13%,75.39%


## Scores by School Type

* Perform the same operations as above, based on school type.

In [52]:
# Replacing the value of type to an integer
school_data_complete["Type"] = school_data_complete["type"].replace({"Charter": 1, "District":2})

# Sample bins. Feel free to create your own bins.
bins = [0, 1, 2]
group_names = ["Charter", "District"]

# Creating the bins in the dataframe
school_data_complete['School Type'] = pd.cut(school_data_complete['Type'], bins, labels = group_names)

# Calculating every column separately
avg_math_score = pd.DataFrame(school_data_complete.groupby('School Type')['math_score'].mean()).rename(columns = {
    'math_score': "Average Math Score" })
avg_reading_score = pd.DataFrame(school_data_complete.groupby('School Type')['reading_score'].mean()).rename(columns = {
    'reading_score': "Average Reading Score" })
per_pass_math = pd.DataFrame(school_data_complete.loc[school_data_complete["math_score"] >= 70,
                                      :].groupby('School Type')["math_score"].count() / 
             school_data_complete.groupby('School Type')["student_name"].count()*100).rename(columns = {
    0: "% Passing Math" })
per_pass_reading = pd.DataFrame(school_data_complete.loc[school_data_complete["reading_score"] >= 70,
                                      :].groupby('School Type')["reading_score"].count() / 
             school_data_complete.groupby('School Type')["student_name"].count()*100).rename(columns = {
    0: "% Passing Reading" })

# Making a join between the columns into dataframe using index as union
score_by_school_type = avg_math_score.join([avg_reading_score,per_pass_math,per_pass_reading])

# Using the first columns of the union to create new ones
score_by_school_type['% Overall Passing Rate'] = (score_by_school_type["% Passing Math"] + score_by_school_type["% Passing Reading"])/2

# Formatting the table
score_by_school_type["Average Math Score"] = score_by_school_type["Average Math Score"].map("{:.2f}".format)
score_by_school_type["Average Reading Score"] = score_by_school_type["Average Reading Score"].map("{:.2f}".format)
score_by_school_type["% Passing Math"] = score_by_school_type["% Passing Math"].map("{:.2f}%".format)
score_by_school_type["% Passing Reading"] = score_by_school_type["% Passing Reading"].map("{:.2f}%".format)
score_by_school_type["% Overall Passing Rate"] = score_by_school_type["% Overall Passing Rate"].map("{:.2f}%".format)

# Deleting index name just for no showing it in the result
del score_by_school_type.index.name

# Showing the result
score_by_school_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Charter,83.41,83.90,93.70%,96.65%,95.17%
District,76.99,80.96,66.52%,80.91%,73.71%
